In [1]:
import jax
import jax.numpy as jnp
from jax import random
from flax import linen as nn
import hard_not
import harden

In [40]:
hnn = hard_not.HardNOT(layer_size=4)

In [18]:
x = [1.0, 1.0]
print(x)

[1.0, 1.0]


In [41]:
weights = hnn.init(random.PRNGKey(0), x)
jax.tree_util.tree_map(lambda x: x.shape, weights) # Checking output shapes

FrozenDict({
    params: {
        weights: (4, 2),
    },
})

In [42]:
y = hnn.apply(weights, x)
print(y)

[[0.         0.59856474]
 [0.         0.        ]
 [0.15280247 0.26057738]
 [0.71123385 0.        ]]
